In [9]:
## Helper functions

import math

def gen_euc_alg(n, m):
    old_r, r = n, m
    old_s, s = 1, 0
    old_t, t = 0, 1

    while r > 0:
        q = old_r // r

        old_r, r = r, old_r - q * r
        old_s, s = s, old_s - q * s
        old_t, t = t, old_t - q * t

    return old_s, old_t, old_r

def is_prime(n):
    i = 2
    while i**2 <= n:
        if n % i == 0:
            return False
        i += 1
    return True


def base_2(n):
    powers = []

    while n > 0: 
        powers.append(n % 2)
        n = n // 2

    return powers
    
def k_power_mod_n(a, k, n):
    _, _, r = gen_euc_alg(a, n)
    if r != 1:
        print(f'(!) gcd({a}, {n}) = {r}')
        return

    # Expand k in base 2
    k_base_2 = base_2(k)

    # Print k as a power of 2s
    k_str = f'{k} = '
    
    for deg in range(len(k_base_2) - 1):
        if k_base_2[deg]: k_str += f'{2**deg} + '
    k_str += f'{2**(len(k_base_2) - 1)}'
    
    # a_powers will store the successive squares of a: a_powers[i] = a^(2^i)
    a_powers = [a]
    outlist = []
    
    # Start the expansion of the result as a product of a^(2^i)
    if k_base_2[0]: 
        ans = a
        outlist.append((a,a))
    else: ans = 1

    # Compute powers of a by successive squaring, use the base 2 expansion of k to evaluate
    for i in range(1, len(k_base_2)):
        a_powers.append(a_powers[i-1]**2 % n)

        if k_base_2[i]: 
            ans = ans * a_powers[i] % n
            outlist.append((f'{a}^{2**i}', a_powers[i]))

    # Return answer as an integer
    return ans

In [11]:
## Solves the discrete log problem g^x = A (mod n) using Shanks' algorithm
## The clever use of a dictionary I stole from ChatGPT

def giant_steps(g, A, n, show_calc=True):
    
    # m > sqrt(n) is needed
    m = int(math.sqrt(n)) + 1
    
    if show_calc: 
        print(f'Using list length m = {m}\n')
    
    # baby steps will hold the entries g, g^2, ..., g^m  
    baby_steps = {}
    current = 1
    
    for j in range(m):
        # the entry is g^j, but the index j is retained via the dictionary
        baby_steps[current] = j
        current = (current * g) % n

    if show_calc:
        print('Baby steps:', sorted(list(baby_steps)))
    
    # Giant steps are of the form Ag^(-im)
    g_inv, _, _ = gen_euc_alg(g, n)

    # step = g^(-im)
    step = k_power_mod_n(g_inv, m, n)
    
    # Initialize giant steps
    giant_step = A
    giant_steps = []
    
    for i in range(m):
        giant_steps.append(giant_step)
        
        if giant_step in baby_steps:
            
            # Solution found: x = j + i * m            
            x = baby_steps[giant_step] + i * m

            if show_calc:
                print('\nGiant steps:', sorted(list(giant_steps)))
                print(f'\nCommon element: {giant_step} = g^{baby_steps[giant_step]} = A * g^(-{i} * m)')
                print(f'x = {baby_steps[giant_step]} + {i} * m = {x}')
                
            return x

        # Iterate through the steps
        giant_step = (giant_step * step) % n

    if show_calc:
        print('Giant steps:', sorted(giant_steps))
        print('\nNo common elements, no solution')
        
    ## If you get to this point then something went wrong. 
    ## It's bad coding but I've set it up so the function returns nothing in this case

In [17]:
### Set these yourself
## n doesn't have to be prime, but the equation may not have solutions for composite n
#g = 58711413111341
#A = 113434134134149
#n = 484511389338941 

g = 20
A = 21

n = 23

##
print(f"Using Shanks' algorithm to solve the discrete log problem g^x = A (mod n)")
print(f'g = {g}')
print(f'A = {A}')
print(f'n = {n} (is prime: {is_prime(n)})')
print() 

x = giant_steps(g=g, A=A, n=n)

if x:   
    print(f'\nSolution found: x = {x}')
    print()
    
    check = k_power_mod_n(a=g, k=x, n=n)
    print(f'Check: {g}^{x} = {A} (mod {n})', check == A)

Using Shanks' algorithm to solve the discrete log problem g^x = A (mod n)
g = 20
A = 21
n = 23 (is prime: True)

Using list length m = 5

Baby steps: [1, 9, 12, 19, 20]

Giant steps: [9, 21]

Common element: 9 = g^2 = A * g^(-1 * m)
x = 2 + 1 * m = 7

Solution found: x = 7

Check: 20^7 = 21 (mod 23) True


In [4]:
# These numbers were used for our ElGamel key, try to crack it using the above
# g, p, A = 90813948194893598391, 567891011121314151617, 497278595931226570035
# Because p is "large" the computation can take a *very* long time


## Keys that work

#g = 58711413111
#A = 113434134
#n = 88888888888889 


In [5]:
y = 201
q = 311

y_list = []

i = 1
Y = y ** 2 % q

while Y != y:
    y_list.append(Y)
    
    Y = Y * y % q
    i += 1

    if i >= q:
        break

print(sorted(y_list))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 18, 20, 21, 24, 25, 26, 27, 28, 30, 32, 35, 36, 39, 40, 42, 45, 47, 48, 49, 50, 52, 53, 54, 56, 60, 63, 64, 65, 67, 70, 72, 73, 75, 78, 79, 80, 81, 83, 84, 89, 90, 91, 94, 96, 98, 100, 104, 105, 106, 107, 108, 109, 112, 113, 117, 120, 121, 125, 126, 127, 128, 130, 134, 135, 137, 139, 140, 141, 144, 146, 147, 150, 156, 157, 158, 159, 160, 162, 163, 166, 168, 169, 173, 175, 178, 179, 180, 182, 187, 188, 189, 192, 193, 195, 196, 197, 200, 208, 209, 210, 212, 214, 216, 218, 219, 223, 224, 225, 226, 229, 234, 235, 237, 240, 242, 243, 245, 249, 250, 252, 253, 254, 256, 260, 265, 267, 268, 270, 273, 274, 277, 278, 280, 282, 288, 289, 292, 294, 300]
